In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [3]:
def dist(a, b):
    return abs(a - b)

In [4]:
def push_forward1(t_mu, mu, phi_c, h):
    assert t_mu.shape == mu.shape == phi_c.shape
    t_mu.fill(0.)                  # 0.で初期化 
    n = phi_c.shape[0]             #  n = \phi_cの要素数

    # map each cell
    for i, mu_val in enumerate(mu):
        if mu_val == 0.:
            continue
        # approximate T_#φ in each corner of the cell
        t_phi = np.zeros((2,))
        # corner indices
        ci = i
        cj = i + 1
        # values of phi_c at the centers of the adjecent cells
        
        """
        -'---'---'---'
        i-1  i  i+1 i+2
         a   b   c   d
        """
        """
        #pattern 1----------
        # セル中心での値を持つ配列から、ci, cjに対応する値を取得する
        # インデックスが負の場合は0に、nを超える場合はn-1にクリップする
        a = phi_c[max(ci - 1, 0): min(ci, n)].flatten()[0]
        b = phi_c[max(cj - 1, 0): min(cj, n)].flatten()[0]
        # compute "x - nabla(phi^c)" using a simple 2nd order central finite difference
        t_phi[0] = ci * h - 0.5 / h * (b - a)         # (b-a)/2h
        t_phi[1] = cj * h - 0.5 / h * (b - a)
        #-----------------------------
        """
        #pattern 2
        a = phi_c[max(ci - 1, 0)]
        b = phi_c[max(ci - 0, 0)]
        c = phi_c[min(cj + 0, n - 1)]
        d = phi_c[min(cj + 1, n - 1)]
        # compute "x - \nabla(phi^c)"  using a simple 2nd order central finite difference
        t_phi[0] = (ci * h) - ((c - a) / (2 * h))
        t_phi[1] = (cj * h) - ((d - b) / (2 * h))

        x_stretch = dist(t_phi[0], t_phi[1])
        x_samples = max(int(np.ceil(x_stretch / h)), 1)  #距離を幅hで割り、小数点以下繰り上げることで,x_stretchの間のグリッドの数を計算.最低グリッド1つ.

        # distribute the cell mass uniformly over all samples
        #輸送前のmuの値を輸送後の場所へ均等に分配するため,輸送後各グリッドのmassを計算
        mass = mu_val / x_samples



        for k in range(x_samples):
            a = (k + 0.5) / x_samples      #a: [0, 1] 

            # sample position in [0,1]
            x = (1. - a) * t_phi[0] + a * t_phi[1]

            # position with respect to cell grid
            xcell = x / h - 0.5

            ti = min(max(int(np.floor(xcell)), 0), n - 1)  #np.floor: 小数点以下切り捨て           
            tio = min(ti + 1, n - 1)

            a = xcell - np.floor(xcell)

            t_mu[ti] += (1. - a) * mass
            t_mu[tio] += a * mass




-'---'---'---' 

$i-1 \, i i+1 i+2 $

 a   b   c   d